### Preparing the 10x data
In this notebook demonstrates how to prepare the Mouse 10x dataset, which is a single-cell dataset was released as part of a transcriptomic cell types study in [Yao et al., 2021](https://portal.brain-map.org/atlases-and-data/rnaseq/mouse-whole-cortex-and-hippocampus-10x). 

In [1]:
import os
import pandas as pd
import requests
import numpy as nps
import anndata as ad
from scipy.sparse import csr_matrix

from mmidas.utils.taxonomy import HTree
from mmidas.utils.tools import logcpm, get_paths, download_file

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

Download the expression matrix and locate them within the ```data/mouse_ctx_10x``` folder. 

In [2]:
toml_file = 'pyproject.toml'
sub_file = '10x_files'
config = get_paths(toml_file=toml_file, sub_file=sub_file)
data_path = config['paths']['main_dir'] / config[sub_file]['data_path']
os.makedirs(data_path, exist_ok=True)


url = 'https://idk-etl-prod-download-bucket.s3.amazonaws.com/aibs_mouse_ctx-hpf_10x/expression_matrix.hdf5' 
download_file(url, data_path / 'mouse_10x.hdf5')

/Users/yeganeh.marghi/github/Dist_MMIDAS/pyproject.toml
Getting files directories belong to 10x_files...


In [3]:
# Load the mouse Smart-seq VISp data
data_VISp_exon = data_path / 'mouse_VISp_2018-06-14_exon-matrix.csv'
anno_VISp = data_path / 'mouse_VISp_2018-06-14_samples-columns.csv'
df_vis_exon = pd.read_csv(data_VISp_exon)
df_vis_anno = pd.read_csv(anno_VISp, encoding='unicode_escape')

# Load the mouse Smart-seq ALM data
data_ALM_exon = data_path / 'mouse_ALM_2018-06-14_exon-matrix.csv'
anno_ALM = data_path / 'mouse_ALM_2018-06-14_samples-columns.csv'
df_alm_exon = pd.read_csv(data_ALM_exon)
df_alm_anno = pd.read_csv(anno_ALM, encoding='unicode_escape')

print(f'Total number of cells in VISp and ALM: {len(df_vis_anno)}, {len(df_alm_anno)}')

Total number of cells in VISp and ALM: 15413, 10068


In [4]:
# Get the neuronal cells across brain regions
vis_neuron = df_vis_anno['class'].isin(['GABAergic', 'Glutamatergic'])
alm_neuron = df_alm_anno['class'].isin(['GABAergic', 'Glutamatergic'])
vis_counts = df_vis_exon.values[:, 1:][:, vis_neuron].T
alm_counts = df_alm_exon.values[:, 1:][:, alm_neuron].T

df_anno_ = pd.concat([df_vis_anno[vis_neuron], df_alm_anno[alm_neuron]], ignore_index=True)
total_count = np.concatenate((vis_counts, alm_counts), axis=0)

# Normalized counts values using LogCPM
logCPM = logcpm(x=total_count, scaler=1e6)
print(np.sum(logCPM, axis=1))

[30890.15859407 34090.13980254 35085.63428565 ... 34077.15380524
 31090.81791427 35629.482184  ]


In [5]:
# list of all genes in the dataset
ref_gene_file = data_path / 'mouse_ALM_2018-06-14_genes-rows.csv'

# selected genes for mouse Smart-seq data analysis
slc_gene_file = data_path / config[sub_file]['ref_gene_file']

ref_genes_df = pd.read_csv(ref_gene_file)
slc_gene_df = pd.read_csv(slc_gene_file)

print(ref_genes_df[41530:41540])
print('-'*100)
print(f'Total number of genes: {len(ref_genes_df)}, Number of selected genes: {len(slc_gene_df)}')

      gene_symbol    gene_id chromosome  gene_entrez_id  \
41530      Sssca1  500741647         19           56390   
41531         Sst  500737291         16           20604   
41532       Sstr1  500729687         12           20605   
41533       Sstr2  500728684         11           20606   
41534       Sstr3  500736064         15           20607   
41535       Sstr4  500704969          2           20608   
41536       Sstr5  500738797         17           20609   
41537       Ssty1  500745186          Y           20611   
41538       Ssty2  500745340          Y           70009   
41539        Ssu2  500714992          6          243612   

                                               gene_name  
41530  Sjogren''s syndrome/scleroderma autoantigen 1 ...  
41531                                       somatostatin  
41532                            somatostatin receptor 1  
41533                            somatostatin receptor 2  
41534                            somatostatin receptor 

Filter out genes that were not selected, as well as two categories of cells: low quality cells, and those belonging to ```CR``` and ```Meis2``` subclasses.

In [6]:
genes = slc_gene_df.genes.values
gene_indx = [np.where(ref_genes_df.gene_symbol.values == gg)[0][0] for gg in genes]
log1p = logCPM[:, gene_indx]

# remove low quality cells and CR and Meis2 subclasses
mask = (df_anno_['cluster']!='Low Quality') & (df_anno_['cluster']!='CR Lhx5') & (df_anno_['cluster']!='Meis2 Adamts19')
df_anno = df_anno_[mask].reset_index() 
log1p = log1p[mask, :]

print(f'final shape of normalized gene expresion matix: {log1p.shape}')

final shape of normalized gene expresion matix: (22365, 5032)


Build an AnnData object for the dataloader. 

In [7]:
# load the tree.csv to obtain colors for t-types on the taxonomies
htree_file = data_path / config[sub_file]['htree_file']
treeObj = HTree(htree_file=htree_file)
ttypes = treeObj.child[treeObj.isleaf]
colors = treeObj.col[treeObj.isleaf]
df_anno.rename(columns={"seq_name": "sample_id", "class": "class_label"})

# rename two cell types according to the taxonomy
df_anno['cluster'][df_anno['cluster'] == 'L6b VISp Col8a1 Rprm'] = 'L6b Col8a1 Rprm'
df_anno['cluster'][df_anno['cluster'] == 'L6 CT ALM Nxph2 Sla'] = 'L6 CT Nxph2 Sla'

In [8]:
# save data as AnnData object
sub_df = df_anno[['sample_name', 'sample_id', 'seq_batch', 'sex', 'brain_hemisphere', 'brain_region', 'brain_subregion', 'class', 'subclass', 'cluster', 'confusion_score']]
adata = ad.AnnData(X=csr_matrix(log1p), obs=sub_df)
adata.var_names = genes
adata.obs_names = sub_df.sample_id.values   
adata.write_h5ad(data_path / config[sub_file]['anndata_file'])